# Kronecker Products of Matrices

The Kronecker product of two matrices, $A \in R^{mxn}$ and $B \in R^{pxq}$ is an $mp$ x $nq$ block matrix, where $A \bigotimes B = \begin{bmatrix} A_{1, 1}\cdot B & \dots & A_{1, n}\cdot B\\ \vdots & \ddots & \vdots\\ A_{m, 1}\cdot B & \dots & A_{m, n}\cdot B \end{bmatrix}$. The following is a parallelized implementation of this operation.

In [1]:
function kronecker(A, B)
    m, n = size(A)
    p, q = size(B)
    
    result = similar(A, p*m, q*n)
    
    #Equivalent to omp parallel for
    Threads.@threads for A_row in 1:m
        for B_row in 1:p
            for A_col in 1:n
                for B_col in 1:q
                    row = p*(A_row-1) + B_row
                    col = q*(A_col-1) + B_col
                    result[row, col] = A[A_row, A_col] * B[B_row, B_col]
                end
            end
        end
    end
    
    result
end

kronecker (generic function with 1 method)

Our environment has access to 8 threads:

In [2]:
Threads.nthreads()

8

Let's generate two random 100x100 matrices.

In [3]:
A = rand(Int, (100, 100)) .% 1000
B = rand(Int, (100, 100)) .% 1000
size(A)

(100, 100)

As expected, the result is a $100^2$x$100^2$ matrix:

In [4]:
size(kronecker(A, B))

(10000, 10000)

# The Power Method

The power method is an iterative method for finding an eigenvalue and eigenvector of a matrix. By repeated application of the matrix on a vector, the vector will eventually converge to an eigenvector corresponding to the eigenvalue of largest magnitude. During this iteration the vector is repeatedly normalized with its 1-norm, which benefits numerical stability.

In [5]:
function power_method(A, max_iterations=100)
    b = rand(eltype(A), size(A)[1])
    
    for _ in 1:max_iterations
        bk = A*b
        # Normalize
        b = bk ./ sum(bk)
    end
    
    b, (A*b)[1] / b[1]
end

power_method (generic function with 2 methods)

Here, we apply this algorithm to a matrix with two real eigenvalues, 8 and 1. 8 has the largest magnitude, so this method will converge to that eigenvalue. The corresponding eigenvector has magnitude 1, but any scalar multiple will also be a valid eigenvector.

In [6]:
A = [3 2; 5 6]
v, l = power_method(A)

([0.28571428571428575, 0.7142857142857143], 7.999999999999998)

We can test that these results do, in fact, satisfy the property of an eigenvector-eigenvalue pair:

In [7]:
A*v

2-element Vector{Float64}:
 2.2857142857142856
 5.714285714285714

In [8]:
l*v

2-element Vector{Float64}:
 2.2857142857142856
 5.7142857142857135

# Jacobi Iteration

Jacobi iteration is a method for solving linear systems $Ax = b$. It relies on the LDU decomposition, which we impliment to begin.

In [9]:
function LDU(A)
    L = zeros(eltype(A), size(A))
    D = zeros(eltype(A), size(A))
    U = zeros(eltype(A), size(A))
    
    n, m = size(A)
    @assert n == m
    
    for row in 1:n
        for col in 1:m
            if row < col
                U[row, col] = A[row, col]
            elseif row > col
                L[row, col] = A[row, col]
            else
                D[row, col] = A[row, col]
            end
        end
    end
    
    L, D, U
end

LDU (generic function with 1 method)

The LDU decomposition simply splits a matrix $A$ into its lower triangular portion $L$, diagonal portion $D$, and upper triangular portion $U$, such that $A = L + D + U$. Consider an arbitrary matrix A:

In [10]:
L, D, U = LDU(A)
A

2×2 Matrix{Int64}:
 3  2
 5  6

After the decomposition, L simply contains the elements below the diagonal:

In [11]:
L

2×2 Matrix{Int64}:
 0  0
 5  0

U contains the elements above the diagonal:

In [12]:
U

2×2 Matrix{Int64}:
 0  2
 0  0

And D contains the diagonal entries:

In [13]:
D

2×2 Matrix{Int64}:
 3  0
 0  6

This decomposition is useful because it gives us access to a diagonal matrix. These are convenient for a number of applications, such as exponentiation. Here, we take advantage of how easy a diagonal matrix is to invert: we can simply invert the diagonal entries.

In [14]:
function diag_inverse(D)
    Dinv = zeros(typeof(1 / D[1, 1]), size(D))
    n = size(D)[1]
    
    for i in 1:n
        Dinv[i, i] = 1 / D[i, i]
    end
    
    Dinv
end

diag_inverse (generic function with 1 method)

In [15]:
D * diag_inverse(D)

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

Jacobi iteration takes advantage of this decomposition. Its main iteration consists of repeatedly applying $D^{-1}(b - (L + U)x)$, to refine the guess $x$.

In [16]:
function jacobi(A, b, max_iterations=100)
    L, D, U = LDU(A)
    Dinv = diag_inverse(D)
    LU = L + U
    
    x = zeros(eltype(A), size(A)[1])
    for _ in 1:max_iterations
        x = Dinv*(b - LU*x)
    end
    
    x
end

jacobi (generic function with 2 methods)